In [1]:
# @title Preparation
#Drive Link materiale: https://drive.google.com/drive/folders/10T3u5qfomOPUeXfHPEzeA3GRiT-ZqgBk?usp=sharing
!pip install tensorflow-gpu==1.15
!pip install scikit-learn==0.20.1

!pip install -q keras-bert keras-rectified-adam
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 411.5MB 39kB/s 
     |████████████████████████████████| 3.8MB 38.6MB/s 
     |████████████████████████████████| 512kB 51.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=759435418dd82c2c702660577922037d613d882b9ac820ebf39e4be3ee08ef71
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc4 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0rc4 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalli

In [2]:
import tensorflow as tf
import keras
from keras_radam import RAdam
from keras_bert import get_custom_objects
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pandas as pd
import tensorflow.keras.backend as K
import sys
from sklearn.metrics import classification_report
from google.colab import drive

Using TensorFlow backend.


In [0]:
# @title Constants
np.random.seed(42)
SEQ_LEN = 128
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5

In [0]:
# @title Environment
import os
pretrained_path = '/content/drive/My Drive/codiesp/'
config_path = os.path.join(pretrained_path, 'config.json')
checkpoint_path = os.path.join(pretrained_path, 'model.ckpt-2000000')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [5]:
# @title Load Basic Model
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
from run_classifier import *
import modeling
import optimization
import tokenization

import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 11.55 MiB/s, done.
Resolving deltas: 100% (185/185), done.



In [6]:
# @title Load Data
!pip install category_encoders==1.3.0
import joblib
import pandas as pd
from keras import Sequential
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import pickle
import numpy as np
import random
from keras.layers import Input
import keras
from keras.layers import Conv1D , Embedding
from keras.layers import Dropout
from keras.layers import MaxPool1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import GlobalMaxPool1D
from keras.layers import Bidirectional
import category_encoders as ce

from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

def remove_symbol(s):
    s = s.replace(",", "")
    s = s.replace(".", "")
    s = s.replace(";", "")
    s = s.replace(":", "")
    s = s.replace("_", "")
    s = s.replace("+", "")
    s = s.replace("ª", "")
    s = s.replace("-", "")
    s = s.replace("<", "")
    s = s.replace(">", "")
    s = s.replace("!", "")
    s = s.replace("?", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    s = s.replace("[", "")
    s = s.replace("]", "")
    s = s.replace("'", "")
    s = s.replace("0", "")
    s = s.replace("1", "")
    s = s.replace("2", "")
    s = s.replace("3", "")
    s = s.replace("4", "")
    s = s.replace("5", "")
    s = s.replace("6", "")
    s = s.replace("7", "")
    s = s.replace("8", "")
    s = s.replace("9", "")
    s = s.replace("%", "")
    s = s.strip()
    s = s.lower()
    return s

# Rimozione delle stopword
def clar_text(text):
    t = remove_symbol(str(text).strip().lower())
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))

    tt = ""
    for it in tokens:
      tt = tt +" "+it
    return tt

def _pad(input_ids, max_seq_len):
    x = []
    input_ids = input_ids[:min(len(input_ids), max_seq_len - 2)]
    input_ids = input_ids + [0] * (max_seq_len - len(input_ids))
    return np.array(input_ids)

#LOADING DATASET
df = pd.read_csv('/content/drive/My Drive/codiesp/TestSet.csv')
df = df[['id', 'Desc']]
# df = df[pd.notnull(df['desc'])]
#print(df.head(10))
print(df.shape)

df.index = range(df.shape[0])
print("Parole: " + str(df['Desc'].apply(lambda x: len(x.split(' '))).sum()))  # ci sono circa 211456 parole

# rimozione SOLO dei simboli (nessuno stemming e nessuna rimozione delle stopword)
df['Desc'] = df['Desc'].apply(remove_symbol)
#print(df.head(10))

# Acquisizione delle stop word
file_stopw = open("/content/drive/My Drive/codiesp/stop_word.pck", "rb")
stop_word = pickle.load(file_stopw)
df['Desc'] = df['Desc'].apply(clar_text)

#suddivisione train_test
test = df

#, test = train_test_split(df, test_size=0.3, random_state=42)

#prepare class encoder
le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
#labels = le.fit(list(df['id']))
mapa = [0,1]

labels_map = [0,1]
#i = 0
#for a in mapa:
#    labels_map.append(a)
#print(labels_map)

#Tokenization
#Inizialize the tokenizer
tokenizer = tokenization.FullTokenizer(vocab_path, do_lower_case=True)
#indices_train = []
indices_test = []

#for text in train['Desc']:
#  tk = tokenizer.tokenize(text)
#  tokens = ["[CLS]"] + tk + ["[SEP]"]
#  token_ids = tokenizer.convert_tokens_to_ids(tokens)
#  token_ids = _pad(token_ids,SEQ_LEN)
#  indices_train.append(token_ids)

for text in test['Desc']:
  tk = tokenizer.tokenize(text)
  tokens = ["[CLS]"] + tk + ["[SEP]"]
  token_ids = tokenizer.convert_tokens_to_ids(tokens)
  token_ids = _pad(token_ids,SEQ_LEN)
  indices_test.append(token_ids)

#indices_train = [indices_train, np.zeros_like(indices_train)]
indices_test= [indices_test, np.zeros_like(indices_test)]
ids = test['id']
print(ids)
#train_labels = train['Code']
#train_labes_indexes = []
#for label in train_labels:
#  if(label =='emp'):
#    train_labes_indexes.append(0)
#  else:
#    train_labes_indexes.append(1)


     |████████████████████████████████| 61kB 2.8MB/s 
(53220, 2)
Parole: 1131000

0        S1137-66272016000300011-1
1        S1137-66272016000300011-1
2        S1137-66272016000300011-1
3        S1137-66272016000300011-1
4        S1137-66272016000300011-1
                   ...            
53215    S1130-01082017000800012-1
53216    S1130-01082017000800012-1
53217    S1130-01082017000800012-1
53218    S1130-01082017000800012-1
53219    S1130-01082017000800012-1
Name: id, Length: 53220, dtype: object


In [0]:
!pip install joblib
import joblib

#labels_map = joblib.load('/content/drive/My Drive/codiesp/labels_map_bert_only_01.joblib')

In [7]:
bert = load_trained_model_from_checkpoint(
    config_file=config_path,
    checkpoint_file=checkpoint_path,
    training=True,
    trainable=True,
    seq_len=128
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# @title Build Custom Model

inputs = bert.inputs[:2]
dense = bert.get_layer('NSP-Dense').output
dense1 = keras.layers.Dense(units=1000, activation='tanh') (dense)
outputs = keras.layers.Dense(units=2, activation='softmax')(dense1)
modelk = keras.models.Model(inputs, outputs)
modelk.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

In [9]:
# @title Initialize Variables
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# @title Fit

#filepath="/content/drive/My Drive/codiesp/task2/bert_only_01.{epoch:05d}-{val_loss:.5f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_sparse_categorical_accuracy', verbose=1, save_best_only=True, mode='max')


#callbacks_list = [checkpoint]

#modelk.fit(
#    indices_train,
#    train_labes_indexes,
#    epochs=50,
#    batch_size=32,
#    validation_split = 0.10,
#    callbacks=callbacks_list
#)

Train on 3880 samples, validate on 432 samples
Epoch 1/50
3880/3880 [==============================] - 76s 20ms/step - loss: 0.6380 - sparse_categorical_accuracy: 0.6232 - val_loss: 0.5574 - val_sparse_categorical_accuracy: 0.7199

Epoch 00001: val_sparse_categorical_accuracy improved from -inf to 0.71991, saving model to /content/drive/My Drive/codiesp/task2/bert_only_01.00001-0.55740.hdf5
Epoch 2/50
3880/3880 [==============================] - 62s 16ms/step - loss: 0.5218 - sparse_categorical_accuracy: 0.7418 - val_loss: 0.4827 - val_sparse_categorical_accuracy: 0.7755

Epoch 00002: val_sparse_categorical_accuracy improved from 0.71991 to 0.77546, saving model to /content/drive/My Drive/codiesp/task2/bert_only_01.00002-0.48266.hdf5
Epoch 3/50
3880/3880 [==============================] - 62s 16ms/step - loss: 0.4371 - sparse_categorical_accuracy: 0.8046 - val_loss: 0.4671 - val_sparse_categorical_accuracy: 0.7894

Epoch 00003: val_sparse_categorical_accuracy improved from 0.77546 to 0

In [0]:
modelk.load_weights('/content/drive/My Drive/codiesp/task2/bert_only_01.00045-1.00324.hdf5')
#modelk.save('/content/drive/My Drive/codiesp/task2/bert_only_01.task2.hdf5')

In [0]:
# @title Predict
predicts = modelk.predict(indices_test, verbose=True)
print(predicts[0])

In [0]:
res_encoded = []
for a in predicts:
    val = a.argmax()
    res_encoded.append(labels_map[val])

print(res_encoded)
#res_labels = le.inverse_transform(DataFrame.res_encoded)

#test_labels = test['Code']
#test_labes_indexes = []
#for label in test_labels:
#  if label == 'emp':
#    test_labes_indexes.append(0)
#  else:
#    test_labes_indexes.append(1)



#print('Testing accuracy %s' % accuracy_score(test_labes_indexes, res_encoded))
#print('Testing F1 score: {}'.format(f1_score(test_labes_indexes, res_encoded, average='weighted')))


In [0]:
!pip install scikit-learn
#from sklearn.metrics import classification_report

In [0]:
print(classification_report(test_labes_indexes, res_encoded,digits=5))

              precision    recall  f1-score   support

           0    0.89598   0.85728   0.87620      1065
           1    0.81665   0.86462   0.83995       783

   micro avg    0.86039   0.86039   0.86039      1848
   macro avg    0.85631   0.86095   0.85807      1848
weighted avg    0.86236   0.86039   0.86084      1848



#CLASSIFICATION pathologies with their numbers


In [0]:
#Estraggo elementi di cui si crede ci sia la patologia
left_to_classify = []
#left_to_classify_labels = []
left_to_classify_ids = []
res_encoded_original = res_encoded.copy()

print(test.head(10))
test_desc = test['Desc']
test_bal = test['id']

print(test_bal[0])
i = 0
for item in res_encoded:
  if item == 1:
    left_to_classify.append(test_desc[i])
    left_to_classify_ids.append(test_bal[i])
  i = i+1

print(len(left_to_classify))
print(len(left_to_classify_ids))

joblib.dump(left_to_classify,'/content/drive/My Drive/codiesp/left_to_classify_Task2.vec')
joblib.dump(left_to_classify_ids,'/content/drive/My Drive/codiesp/left_to_classify_ids_Task2.vec')


                          id                                               Desc
0  S1137-66272016000300011-1   varón  años consultó dolor tumefacción borde ...
1  S1137-66272016000300011-1                     refería antecedente traumático
2  S1137-66272016000300011-1        dolor carácter espontáneo inicio progresivo
3  S1137-66272016000300011-1   dicho dolor inicialmente cedía aine increment...
4  S1137-66272016000300011-1   exploración física apreció tumefacción mal de...
5  S1137-66272016000300011-1                 enrojecimiento aumento calor local
6  S1137-66272016000300011-1                  movilidad muñeca dedos conservada
7  S1137-66272016000300011-1   radiografía simple mostraba lesión ósea radio...
8  S1137-66272016000300011-1   completó estudio gammagrafía ósea tac resonan...
9  S1137-66272016000300011-1   gammagrafía ósea tres fases realizada tecneci...
S1137-66272016000300011-1
13253
13253


['/content/drive/My Drive/codiesp/left_to_classify_ids_Task2.vec']

In [0]:
# Tokenizzazione e rimozione delle stopword
def tokenize_text(text):
    tokens = list(str(text).lower().split(" "))
    for z in range(0, len(stop_word)):
        if stop_word[z] in tokens:
            while stop_word[z] in tokens:
                tokens.remove(str(stop_word[z]))
    return tokens

In [0]:
# prepare tokenizer
#t = Tokenizer()
#t.fit_on_texts(df['Desc'])
#vocab_size = len(t.word_index) + 1

t = joblib.load("/content/drive/My Drive/codiesp/task2/lstm_fasttext__tokenizer_total_Task2.vec")

In [0]:
#padding text
encoded_left_to_classify = t.texts_to_sequences(left_to_classify)

max_length = 64
padded_left_to_classify = pad_sequences(encoded_left_to_classify, maxlen=max_length, padding='post')
print(padded_left_to_classify[76])

[  35 1255  131   43 2331  341  901  237 1494 6188    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


In [0]:
#le = ce.OneHotEncoder(return_df=False, impute_missing=False, handle_unknown="ignore")
#labels = le.fit(list(df['Code']))
le = joblib.load("/content/drive/My Drive/codiesp/task2/lstm_fasttext__label_encoder_le_Task2.vec")

In [0]:
# get the vectors
#file = open('/content/drive/My Drive/codiesp/embeddings-l-model_es.vec')

# create a weight matrix for words in training docs
#count = 0
embedding_matrix = joblib.load('/content/drive/My Drive/codiesp/task2/lstm_fasttext__embedding_matrix_medical_total_task2.vec')
#np.zeros((vocab_size, 300))
#vocab_and_vectors = {}
#arrValues = []
#z = 0
#for line in file:
#    if(z != 0):
#        values = line.split()
#        word = values[0]
#        vector = np.asarray(values[1:], dtype='float32')
#        vocab_and_vectors[word] = vector
#        arrValues.append(vector)
#    else:
#        z= z+1

##for word, i in t.word_index.items():
#    try:
#        embedding_vector = vocab_and_vectors.get(word)#
#
#        if embedding_vector is None:
#            count = count + 1
#            index = random.randint(0, 1000)
#            embedding_vector = arrValues[index]
#    except:
#        count = count+1
#        index = random.randint(0, 1000)
#        embedding_vector = arrValues[index]

#    if embedding_vector is not None:
#        embedding_matrix[i] = embedding_vector
#print(count)

In [0]:
# define the model
vocab_size = len(t.word_index) + 1
input = Input(shape=(64,))
m = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=64, trainable=False) (input)
bi = Bidirectional(LSTM(64, activation ='tanh', return_sequences = True, dropout=0.3)) (m)

aa = SeqSelfAttention(attention_activation='tanh') (bi)
aa = Conv1D(128,5, activation ='relu' ) (aa)
aa = MaxPool1D(2) (aa)
aa = Dropout(0.2) (aa)

added = keras.layers.Concatenate(axis=1)([aa,bi])

ff = GlobalMaxPool1D() (added)
ff = Dense(4000)(ff)
ff = Dropout(0.3) (ff)
ff =Dense(1788, activation='softmax') (ff)

model = keras.models.Model(inputs=[input], outputs=[ff])

model.summary(line_length=100)
model.compile (loss='categorical_crossentropy' , optimizer='adam' , metrics=[ 'accuracy'] )
model.load_weights('/content/drive/My Drive/codiesp/task2/LSTM_CNN_ATT_Fasttext_03052020weights.00029-4.52123.hdf5')



Model: "model_3"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 64, 300)       1880700     input_1[0][0]                    
____________________________________________________________________________________________________
bidirectional_1 (Bidirectional)  (None, 64, 128)       186880      embedding_1[0][0]                
____________________________________________________________________________________________________
seq_self_attention_1 (SeqSelfAtt (None, 64, 128)       8257        bidirectional_1[0][0]            
_________________________________________________________________________

In [0]:

res = model.predict(padded_left_to_classify)
joblib.dump(res, '/content/drive/My Drive/codiesp/res_Task2.vec')
print(res[0])
#map = le.category_mapping[0]['mapping']
#print(map)
#print(res_encoded)

#labels_map = []
#i = 0
#for a,b in map:
#    labels_map.append(a)

#res_encoded = []
#for a in res:

#    val = a.argmax()
#    res_encoded.append(labels_map[val])

#print(left_to_classify_labels)

[7.05051527e-04 4.97213099e-04 1.56527048e-03 6.04105589e-04
 3.16707738e-05 2.91531370e-03 1.68417842e-04 7.47114609e-05
 3.56424498e-05 9.60288889e-05 1.08789105e-03 1.60243188e-04
 1.38589265e-02 8.05002230e-04 1.10787487e-05 2.02795910e-03
 5.54815400e-03 5.03839692e-04 3.51085764e-04 4.40641277e-04
 7.42751945e-05 5.44132607e-04 8.92326643e-04 9.12478019e-04
 7.03163960e-06 1.40397942e-05 9.06575297e-05 5.73063083e-03
 3.41862417e-03 5.10056212e-04 8.16492684e-05 2.92362995e-04
 5.71983983e-04 3.29730901e-05 1.67266268e-03 2.95853417e-04
 2.46175099e-03 2.50093261e-04 1.52443186e-04 3.43923020e-04
 9.98855321e-05 6.94567338e-04 3.85171479e-05 2.62335781e-03
 1.47469167e-04 6.05594323e-05 4.87501238e-04 3.31651208e-05
 8.54277823e-05 3.77290075e-06 7.70590559e-04 9.10529525e-06
 6.89606939e-04 4.43525023e-05 1.11342780e-03 1.03963132e-03
 2.74321355e-04 1.49649233e-04 1.87359954e-04 4.87134566e-05
 9.08214017e-04 2.50983885e-05 9.59702957e-05 5.25486284e-05
 1.39744196e-04 6.501201

In [0]:
from collections import OrderedDict 
max_vals={}
extracted_res = []
extractede_ids = []
map = le.category_mapping[0]['mapping']
labels_map = []
i = 0
for a,b in map:
    labels_map.append(a)

j =0
previous_id = left_to_classify_ids[0]
for result in res:
  id = left_to_classify_ids[j]

  if id != previous_id:
    
    od = OrderedDict(max_vals) 
    #print(str(id)+' '+str(od))

    for key, value in od.items():
      extracted_res.append(value.lower())
      extractede_ids.append(previous_id)
    max_vals = {}
    previous_id = id

  i = 0
  for a in result:
    if a > 0.10:
      max_vals[str(a)] = labels_map[i]
    i = i+1
  if len(max_vals) == 0:
    v = result.max()
    index = result.argmax()
    max_vals[str(v)] = labels_map[index]

  j = j+1

#Last item
od = OrderedDict(max_vals) 

for key, value in od.items():
    extracted_res.append(value.lower())
    extractede_ids.append(previous_id)

print(extracted_res)
print(extractede_ids)

['4a07x0z', 'b926', 'bn06zzz', 'b030', 'b020', '3e0t3cz', 'cp1yyzz', '4a0f', 'bw2f', 'bn23', 'bn26', '0kxj', '0016', '0s9d', '0nbv', '0cb7', '0sj94zz', '10e0xzz', '10d0', 'bn0hzzz', '0b11', '0nb0', '0cb7', 'bw03zzz', 'b246zzz', '0fb0', 'bb24', '5a1d', '0ft4', '0fb0', '0fb03zx', 'cp1z', 'bt43zzz', '0dj08zz', 'bw0lzzz', '0fb0', '0db9', '0tbb8zz', 'bw03zzz', 'bw00zzz', '0t9b30z', '0dj08zz', 'bt20', '0fy0', '3e0233z', '0gtk', '0dj68zz', 'bd42zzz', 'bp0l', '0nbt', 'bn06zzz', '3e05305', 'cg121zz', 'bw40zzz', '0ft4', 'bw20', 'bw24', '0bbk', '0bbn4zx', 'bw20', '0vtg', '0dtb', 'b52k', '0vtb', '0gtk', 'bw20', 'bw20', 'bw20', 'bw00zzz', 'bn0jzzz', '0016', '3e0r3cz', 'h47.10', '0w993zz', 'bw21', 'bw24', 'bn26', '0nst', '07t1', '4a02x4z', '4a00x4z', 'b236', 'b040', 'bw40zzz', '0fb0', '0dj68zz', '0db9', '0fbg3zx', '0vb0', '0vt00zz', '0vb0', 'b030', 'b020', '0w99', 'b030', 'b030yzz', 'bt23', 'b246zzz', '0fb0', '0fb24zz', '0vb0', '0gbr', '0gbn', '0yq5', '0ft4', '5a1d', '3e1m39z', '0dj08zz', 'bn20', 'b

In [0]:
file = open('0.task2_run1_bert_01_multiclass_deep_net.tsv','w+')

for i in range(0, len(extracted_res)):
  file.write(str(extractede_ids[i])+'\t'+str(extracted_res[i])+'\n')
  file.flush()

file.close()

In [0]:
print('Testing accuracy %s' % accuracy_score(left_to_classify_labels, res_encoded))
print('Testing F1 score: {}'.format(f1_score(left_to_classify_labels, res_encoded, average='weighted')))

from sklearn.metrics import classification_report
#print(classification_report(left_to_classify_labels, res_encoded,digits=5))

Testing accuracy 0.5247285886610374
Testing F1 score: 0.4693915916051327


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


#Merge back the results


In [0]:
final_res = []

i = 0
for item in res_encoded_original:
  if item == 0:
    final_res.append('emp')
  else:
    final_res.append(res_encoded[i])
    i = i+1


In [0]:
print('Testing accuracy %s' % accuracy_score(test_bal, final_res))
print('Testing F1 score: {}'.format(f1_score(test_bal, final_res, average='weighted')))
print(classification_report(test_bal, final_res,digits=5))

Testing accuracy 0.7294372294372294
Testing F1 score: 0.7237277662975383
              precision    recall  f1-score   support

        0016    0.50000   0.50000   0.50000         2
     00BM3ZX    0.00000   0.00000   0.00000         0
        00b1    0.00000   0.00000   0.00000         1
     00b20zz    0.00000   0.00000   0.00000         0
        00c0    1.00000   1.00000   1.00000         1
     00u10jz    0.00000   0.00000   0.00000         1
        0210    0.00000   0.00000   0.00000         1
        0212    0.50000   1.00000   0.66667         1
        0213    0.00000   0.00000   0.00000         0
        02rg    0.50000   0.50000   0.50000         2
     02rg0jz    0.00000   0.00000   0.00000         1
     02vw3dz    1.00000   1.00000   1.00000         1
        0318    1.00000   1.00000   1.00000         1
        03bs    0.00000   0.00000   0.00000         1
     04l33dz    0.00000   0.00000   0.00000         3
     06hm33z    0.00000   0.00000   0.00000         0
        

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
